In [267]:
#Fav / Dog Win percentage + odds filter
import pandas as pd
from sqlalchemy import create_engine
import pandas as pd
import numpy as np
import datetime
stake=100
datefilter='2023-01-01'

devengine = create_engine("sqlite:///C:/Git/tennis_atp/database/bets_sqllite.db")
time_now = datetime.datetime.now()

def hist_data_dog(surface, sex, odds_low,odds_high,date):
    data=pd.read_sql_query(f"Select distinct * from results_{surface}_1 where date > '{date}' ",con=devengine)#.to_clipboard(index=False)
    data["Fav_Odds"] = data.Fav_Odds.astype(float)
    data["Dog_Odds"] = data.Dog_Odds.astype(float)
    data['Fav_Win'] = data['Winner']==data['Fav']
    data['DogWin'] = data['Winner']!=data['Fav']
    data=data[(data['Fav_Odds']>=odds_low)&(data['Fav_Odds']<=odds_high)]
    data=data[data['Sex']==sex]
    return data

In [268]:
clay_womens=hist_data_dog('Clay','Womens',1,2,datefilter)
clay_mens=hist_data_dog('Clay','Mens',1,2,datefilter)
temp=pd.concat([clay_womens,clay_mens]).sort_values(by='Date')
temp = temp.reset_index(drop=True)
temp['Fav_Loss'] = (temp['Fav_Win'] == 0).astype(int)
temp['Fav_Loss_Count'] = temp.groupby('Fav')['Fav_Loss'].cumsum()
temp['Dog_Win'] = (temp['Fav_Win'] == 0).astype(int)
temp['Dog_Win_Count'] = temp.groupby('Dog')['Dog_Win'].cumsum()
mind=temp[temp['Fav_Odds']<1.3].copy()
mind=mind[mind['Fav_Odds']>1]
mind=mind[mind['Date']>='2024-01-01']

favs=pd.read_csv('Fav_Hard.csv')
fav_list=favs['Name'].tolist()
mind=mind[(mind['Fav']).isin(fav_list)]
mind=mind[mind['dog_rank']>50]
len(mind[mind['Fav']==mind['Winner']])/len(mind),len(mind)
mind[['Fav','Dog','Fav_Odds','Winner','Fav_Win','dog_rank']].to_clipboard(index=False)



C:\Users\Administrator\AppData\Local\Temp\ipykernel_8856\4290647011.py:34: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(wins_last_30_days)


In [324]:
calc=temp.copy()
import pandas as pd

# Ensure 'Date' is a datetime type
calc['Date'] = pd.to_datetime(calc['Date'])

# Melt Fav and Dog results into long format for players and their wins
fav_df = calc[['Date', 'Fav', 'Fav_Win']].rename(columns={'Fav': 'Player', 'Fav_Win': 'Win'})
dog_df = calc[['Date', 'Dog', 'Dog_Win']].rename(columns={'Dog': 'Player', 'Dog_Win': 'Win'})

# Concatenate both datasets
player_results = pd.concat([fav_df, dog_df], ignore_index=True)

# Keep only the rows where there was a win
player_wins = player_results[player_results['Win'] == True].copy()

# Sort player wins by Player and Date
player_wins = player_wins.sort_values(['Player', 'Date'])

# Function to count wins in the last 30 days, excluding the current match
def wins_last_30_before_match(group):
    result = []
    for i in range(len(group)):
        cutoff = group.iloc[i]['Date']
        # Filter for wins that happened before the cutoff date within the last 30 days
        past_wins = group[(group['Date'] < cutoff) & (group['Date'] >= cutoff - pd.Timedelta(days=30))]
        result.append(len(past_wins))  # Count the wins in the past 30 days
    return pd.Series(result, index=group.index)

# Calculate wins in the last 30 days for each player before the match
player_wins['Wins_Last_30_Days_Before'] = (
    player_wins.groupby('Player', group_keys=False)
    .apply(wins_last_30_before_match)
)

# Merge the calculated win counts back into the original dataframe

# Sort for merge_asof (allows for the "before match" logic)
calc = calc.sort_values('Date')
player_wins = player_wins.sort_values('Date')

# Merge back for Fav (before the match date)
fav_win_counts = player_wins[['Player', 'Date', 'Wins_Last_30_Days_Before']].rename(columns={
    'Player': 'Fav',
    'Date': 'Match_Date',
    'Wins_Last_30_Days_Before': 'Fav_Wins_Last_30_Days_Before'
})

calc = pd.merge_asof(
    calc.sort_values('Date'),
    fav_win_counts.sort_values('Match_Date'),
    by='Fav',
    left_on='Date',
    right_on='Match_Date',
    direction='backward'
).drop(columns=['Match_Date'])

# Merge back for Dog (before the match date)
dog_win_counts = player_wins[['Player', 'Date', 'Wins_Last_30_Days_Before']].rename(columns={
    'Player': 'Dog',
    'Date': 'Match_Date',
    'Wins_Last_30_Days_Before': 'Dog_Wins_Last_30_Days_Before'
})

calc = pd.merge_asof(
    calc.sort_values('Date'),
    dog_win_counts.sort_values('Match_Date'),
    by='Dog',
    left_on='Date',
    right_on='Match_Date',
    direction='backward'
).drop(columns=['Match_Date'])

# Fill any missing win counts with 0 (if no prior wins)
calc['Fav_Wins_Last_30_Days_Before'] = calc['Fav_Wins_Last_30_Days_Before'].fillna(0).astype(int)
calc['Dog_Wins_Last_30_Days_Before'] = calc['Dog_Wins_Last_30_Days_Before'].fillna(0).astype(int)


C:\Users\Administrator\AppData\Local\Temp\ipykernel_8856\74366135.py:33: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(wins_last_30_before_match)


In [327]:
calc[(calc['Fav']=='Petr Brunclik')|(calc['Dog']=='Petr Brunclik')][['Date','Winner','Fav','Dog','Fav_Wins_Last_30_Days_Before','Dog_Wins_Last_30_Days_Before']]
#calc[(calc['Fav']=='Francesca Jones')|(calc['Dog']=='Francesca Jones')][['Date','Winner','Fav','Dog','Fav_Wins_Last_30_Days_Before','Dog_Wins_Last_30_Days_Before']]
	

,Date,Winner,Fav,Dog,Fav_Wins_Last_30_Days_Before,Dog_Wins_Last_30_Days_Before
6778,2025-05-03,Gauthier Onclin,Gauthier Onclin,Petr Brunclik,1,0
6867,2025-05-06,Petr Brunclik,Petr Brunclik,Shintaro Mochizuki,0,1
6936,2025-05-07,Maxime Chazal,Petr Brunclik,Maxime Chazal,0,3
7951,2025-06-02,Petr Brunclik,Petr Brunclik,Gerard Campana Lee,1,0
7956,2025-06-03,Alexander Shevchenko,Alexander Shevchenko,Petr Brunclik,7,1


In [299]:
calc['Predicted_Winner'] = calc['Fav_Wins_Last_30_Days_Before'] < calc['Dog_Wins_Last_30_Days_Before']
accuracy = (calc['Predicted_Winner'] == calc['Dog_Win']).mean()
print(f"Accuracy: {accuracy:.2%}")


Accuracy: 62.14%


In [303]:
stake=100
date_filter=calc[calc['Date']>'2025-01-01']
oddsfilter=date_filter[date_filter['Fav_Odds']>1.4].copy()
oddsfilter=oddsfilter[oddsfilter['Dog_Wins_Last_30_Days_Before']>oddsfilter['Fav_Wins_Last_30_Days_Before']]
oddsfilter=oddsfilter[oddsfilter['Sex']=='Womens']
oddsfilter['Profit'] = oddsfilter.apply(
    lambda row: (row['Dog_Odds'] - 1) * stake if row['Winner'] == row['Dog'] else -stake, axis=1
)

# Calculate total profit
total_profit = oddsfilter['Profit'].sum()

# Print the total profit
print("Total Profit:", total_profit, len(oddsfilter),len(oddsfilter[oddsfilter['Dog']==oddsfilter['Winner']])/len(oddsfilter))
oddsfilter[['Date','Dog_Odds','Winner','Dog','Fav','Dog_Win_Count','Fav_Loss_Count','Dog_Wins_Last_30_Days_Before','Fav_Wins_Last_30_Days_Before']]
oddsfilter#.to_csv('_ClayResults_Womens.csv')

Total Profit: 2936.0 159 0.5283018867924528


,Date,Winner_Odds,Winner,Fav_Odds,Dog_Odds,Fav,Elo_Fav,Fav_Games,Dog,Dog_Games,...,Fav_Win,DogWin,Fav_Loss,Fav_Loss_Count,Dog_Win,Dog_Win_Count,Fav_Wins_Last_30_Days_Before,Dog_Wins_Last_30_Days_Before,Predicted_Winner,Profit
5372,2025-03-18,1.44,Alexia Todoni Anca,1.44,2.71,Alexia Todoni Anca,Alexia Todoni Anca,18,Panna Udvardy,11,...,True,False,0,2,0,4,0,6,True,-100.0
5439,2025-03-20,1.54,Clara Burel,1.54,2.41,Clara Burel,Clara Burel,12,Darja Semenistaja,21,...,True,False,0,2,0,6,1,5,True,-100.0
5450,2025-03-21,2.24,Leyre Romero Gormaz,1.61,2.24,Victoria Jimenez Kasintseva,Victoria Jimenez Kasintseva,10,Leyre Romero Gormaz,11,...,False,True,1,2,1,3,0,1,True,124.0
5475,2025-03-22,2.15,Leyre Romero Gormaz,1.66,2.15,Maja Chwalinska,Maja Chwalinska,10,Leyre Romero Gormaz,11,...,False,True,1,1,1,4,1,2,True,115.0
5514,2025-03-24,1.67,Tatiana Prozorova,1.67,2.13,Tatiana Prozorova,Tamara Zidansek,9,Tamara Zidansek,11,...,True,False,0,1,0,3,0,1,True,-100.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8560,2025-07-07,1.51,Jule Niemeier,1.51,2.50,Jule Niemeier,Nao Hibino,10,Nao Hibino,13,...,True,False,0,5,0,4,0,2,True,-100.0
8600,2025-07-09,1.57,Antonia Ruzic,1.57,2.32,Antonia Ruzic,Mona Barthel,10,Mona Barthel,12,...,True,False,0,2,0,6,1,2,True,-100.0
8650,2025-07-10,2.1,Antonia Ruzic,1.70,2.10,Lucia Bronzetti,Antonia Ruzic,11,Antonia Ruzic,10,...,False,True,1,5,1,2,0,2,True,110.0
8674,2025-07-12,1.56,Francesca Jones,1.56,2.36,Francesca Jones,Francesca Jones,11,Petra Marcinko,13,...,True,False,0,1,0,4,2,3,True,-100.0


In [275]:
stake=100
date_filter=calc[calc['Date']>'2025-07-05']
oddsfilter=date_filter[date_filter['Fav_Odds']>1.7].copy()
oddsfilter=oddsfilter[oddsfilter['Dog_Wins_Last_30_Days_Before']>oddsfilter['Fav_Wins_Last_30_Days_Before']]
oddsfilter=oddsfilter[oddsfilter['Sex']=='Mens']
oddsfilter['Profit'] = oddsfilter.apply(
    lambda row: (row['Dog_Odds'] - 1) * stake if row['Winner'] == row['Dog'] else -stake, axis=1
)

# Calculate total profit
total_profit = oddsfilter['Profit'].sum()

# Print the total profit
print("Total Profit:", total_profit, len(oddsfilter),len(oddsfilter[oddsfilter['Dog']==oddsfilter['Winner']])/len(oddsfilter))
oddsfilter[['Date','Dog_Odds','Winner','Dog','Fav','Dog_Wins_Last_30_Days_Before','Fav_Wins_Last_30_Days_Before']]
#oddsfilter#.to_csv('_ClayResults_Mens.csv')

Total Profit: 284.0 9 0.6666666666666666


,Date,Dog_Odds,Winner,Dog,Fav,Dog_Wins_Last_30_Days_Before,Fav_Wins_Last_30_Days_Before
8517,2025-07-06,1.95,Thiago Seyboth Wild,Thiago Seyboth Wild,Jerome Kym,3,1
8527,2025-07-06,1.92,Daniel Masur,Roberto Cid,Daniel Masur,1,0
8529,2025-07-06,2.00,Francesco Maestrelli,Francesco Maestrelli,Luka Pavlovic,6,3
8532,2025-07-07,2.00,Branko Djuric,Adrian Boitan Gabi,Branko Djuric,3,0
8540,2025-07-07,1.97,Rudolf Molleker,Gonzalo Bueno,Rudolf Molleker,4,2
8626,2025-07-09,1.96,Stefano Travaglia,Stefano Travaglia,Tristan Boyer,8,1
8634,2025-07-09,1.92,Francesco Maestrelli,Francesco Maestrelli,Pablo Llamas Ruiz,7,1
8654,2025-07-10,1.97,Titouan Droguet,Titouan Droguet,Hugo Dellien,4,0
8665,2025-07-11,2.04,Stefano Travaglia,Stefano Travaglia,Vitaliy Sachko,8,0


In [276]:
stake=100
date_filter=calc[calc['Date']>'2025-07-01']
oddsfilter=date_filter[date_filter['Fav_Odds']>1.8].copy()
oddsfilter=oddsfilter[oddsfilter['Dog_Wins_Last_30_Days_Before']<oddsfilter['Fav_Wins_Last_30_Days_Before']]
oddsfilter=oddsfilter[oddsfilter['Sex']=='Mens']
oddsfilter['Profit'] = oddsfilter.apply(
    lambda row: (row['Fav_Odds'] - 1) * stake if row['Winner'] == row['Fav'] else -stake, axis=1
)

# Calculate total profit
total_profit = oddsfilter['Profit'].sum()

# Print the total profit
print("Total Profit:", total_profit, len(oddsfilter),len(oddsfilter[oddsfilter['Fav']==oddsfilter['Winner']])/len(oddsfilter))
oddsfilter[['Date','Dog_Odds','Winner','Dog','Fav','Dog_Win_Count','Fav_Loss_Count','Dog_Wins_Last_30_Days_Before','Fav_Wins_Last_30_Days_Before']]
#oddsfilter.to_csv('_ClayResults_Mens.csv')

Total Profit: -300 3 0.0


,Date,Dog_Odds,Winner,Dog,Fav,Dog_Win_Count,Fav_Loss_Count,Dog_Wins_Last_30_Days_Before,Fav_Wins_Last_30_Days_Before
8469,2025-07-02,1.87,Murkel Dellien,Murkel Dellien,Radu Albot,12,8,0,2
8575,2025-07-08,1.89,Francesco Passaro,Francesco Passaro,Luca Van Assche,16,11,5,6
8669,2025-07-11,1.91,Sumit Nagal,Sumit Nagal,Marco Cecchinato,9,15,3,10


In [277]:
from sqlalchemy import text

#with devengine.connect() as conn:
#    conn.execute(text('DELETE FROM results_clay_1 WHERE Date > "2025-06-07"'))
#    conn.commit()
#pd.read_sql_query('Select distinct* from AllMatches where date = "2025-06-08" and Sex like "Womens"',con=devengine)

In [318]:
tempo = temp.copy()
date_filter = pd.read_pickle(r'.\Clay_Today').drop(columns=['Fav_Loss_Count','Dog_Win_Count'])
date_filter['Date'] = '2025-07-13'
calc=pd.concat([tempo, date_filter], ignore_index=True)


In [ ]:

# Ensure 'Date' is datetime
calc['Date'] = pd.to_datetime(calc['Date'])

# Melt data into long format
fav_df = calc[['Date', 'Fav', 'Fav_Win']].rename(columns={'Fav': 'Player', 'Fav_Win': 'Win'})
dog_df = calc[['Date', 'Dog', 'Dog_Win']].rename(columns={'Dog': 'Player', 'Dog_Win': 'Win'})

# Combine both
player_results = pd.concat([fav_df, dog_df], ignore_index=True)

# Filter for wins only
player_wins = player_results[player_results['Win'] == True].copy()

# Sort by player and date
player_wins = player_wins.sort_values(['Player', 'Date'])

# Rolling count of past 30 days wins
def wins_last_30_days(group):
    result = []
    for i in range(len(group)):
        end_date = group.iloc[i]['Date']
        start_date = end_date - pd.Timedelta(days=30)
        # Count wins within the past 30 days (excluding current match)
        past_wins = group[(group['Date'] < end_date) & (group['Date'] > start_date)]
        result.append(len(past_wins))
    return pd.Series(result, index=group.index)

player_wins['Wins_Last_30_Days'] = (
    player_wins.groupby('Player', group_keys=False)
    .apply(wins_last_30_days)
)

# Merge back into main DataFrame
# Prepare win counts for Fav
fav_win_counts = player_wins[['Player', 'Date', 'Wins_Last_30_Days']].rename(columns={
    'Player': 'Fav',
    'Date': 'Match_Date',
    'Wins_Last_30_Days': 'Fav_Wins_Last_30_Days'
})

calc = pd.merge_asof(
    calc.sort_values('Date'),
    fav_win_counts.sort_values('Match_Date'),
    by='Fav',
    left_on='Date',
    right_on='Match_Date',
    direction='backward'  # Because it's past wins
).drop(columns=['Match_Date'])

# Prepare win counts for Dog
dog_win_counts = player_wins[['Player', 'Date', 'Wins_Last_30_Days']].rename(columns={
    'Player': 'Dog',
    'Date': 'Match_Date',
    'Wins_Last_30_Days': 'Dog_Wins_Last_30_Days'
})

calc = pd.merge_asof(
    calc.sort_values('Date'),
    dog_win_counts.sort_values('Match_Date'),
    by='Dog',
    left_on='Date',
    right_on='Match_Date',
    direction='backward'
).drop(columns=['Match_Date'])

# Fill missing values with 0
calc['Fav_Wins_Last_30_Days'] = calc['Fav_Wins_Last_30_Days'].fillna(0).astype(int)
calc['Dog_Wins_Last_30_Days'] = calc['Dog_Wins_Last_30_Days'].fillna(0).astype(int)





C:\Users\Administrator\AppData\Local\Temp\ipykernel_8856\3255646066.py:30: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(wins_last_30_days)


,Date,Winner_Odds,Winner,Fav_Odds,Dog_Odds,Fav,Elo_Fav,Fav_Games,Dog,Dog_Games,...,Dog_Record,Resulted,Fav_Serve%,Fav_Return%,Dog_Serve%,Dog_Return%,fav_last_five_win_perc,dog_last_five_win_perc,Fav_Wins_Last_30_Days,Dog_Wins_Last_30_Days
8691,2025-07-13,NaN,NaN,1.41,2.81,Calvin Hemery,Facundo Bagnis,1,Facundo Bagnis,14,...,93%,False,0.00,0.00,0.73,0.22,0.8,0.400000,3,0
8692,2025-07-13,NaN,NaN,1.31,3.15,Thiago Seyboth Wild,Thiago Seyboth Wild,7,Nicolai Budkov Kjaer,4,...,50%,False,0.68,0.17,NaN,NaN,0.8,0.600000,3,1
8693,2025-07-13,NaN,NaN,1.34,3.11,Rodrigues Alves Carolina Meligeni,Dalina Amariei Ilinca,5,Dalina Amariei Ilinca,2,...,50%,False,NaN,NaN,NaN,NaN,0.2,0.333333,0,0
8694,2025-07-13,NaN,NaN,1.55,2.33,Oleg Prihodko,Oleg Prihodko,10,Gianluca Mager,13,...,46%,False,0.00,0.00,0.80,0.10,0.6,0.200000,3,1
8695,2025-07-13,NaN,NaN,1.45,2.60,Carlos Prado Angelo Juan,Carlos Prado Angelo Juan,12,Stefanos Sakellaridis,3,...,33%,False,NaN,NaN,NaN,NaN,0.8,0.600000,4,3
8696,2025-07-13,NaN,NaN,1.12,6.20,Henrique Rocha,Henrique Rocha,13,Patrick Zahraj,2,...,0%,True,NaN,NaN,NaN,NaN,0.4,0.200000,5,0
8697,2025-07-13,NaN,NaN,1.38,2.89,Mariano Navone,Juan Manuel Cerundolo,36,Juan Manuel Cerundolo,30,...,40%,True,0.00,0.00,0.65,0.20,0.8,0.800000,3,5
8698,2025-07-13,NaN,NaN,1.77,1.96,Thiago Agustin Tirante,Thiago Agustin Tirante,28,Facundo Diaz Acosta,19,...,37%,True,0.00,0.00,0.00,0.00,0.8,0.800000,6,3
8699,2025-07-13,NaN,NaN,1.58,2.15,Neil Oberleitner,Neil Oberleitner,6,Anton Matusevich,4,...,0%,False,NaN,NaN,NaN,NaN,0.6,0.200000,1,0
8700,2025-07-13,NaN,NaN,1.65,2.18,Elsa Jacquemot,Oksana Selekhmeteva,15,Oksana Selekhmeteva,12,...,42%,True,0.46,0.36,0.48,0.40,0.8,0.800000,2,2


In [321]:
today=calc[calc['Date']=='2025-07-13']
today=today[today['Fav_Odds']>1.4]
today=today[today['Dog_Wins_Last_30_Days']>today['Fav_Wins_Last_30_Days']]
womens=today[today['Sex']=='Womens']
womens[['Time','Dog','Dog_Odds','Fav','Dog_Wins_Last_30_Days','Fav_Wins_Last_30_Days']]


,Time,Dog,Dog_Odds,Fav,Dog_Wins_Last_30_Days,Fav_Wins_Last_30_Days


In [322]:
today=calc[calc['Date']=='2025-07-13']
today=today[today['Fav_Odds']>1.7]
today=today[today['Dog_Wins_Last_30_Days']>today['Fav_Wins_Last_30_Days']]
mens=today[today['Sex']!='Womens']
mens[['Time','Dog','Dog_Odds','Fav','Fav_Odds','Dog_Wins_Last_30_Days','Fav_Wins_Last_30_Days']]

,Time,Dog,Dog_Odds,Fav,Fav_Odds,Dog_Wins_Last_30_Days,Fav_Wins_Last_30_Days
8708,23:00,Jelle Sels,1.87,Petr Brunclik,1.78,2,1


In [280]:
date_filter = pd.read_pickle(r'.\Clay_Today').drop(columns=['Fav_Loss_Count','Dog_Win_Count'])
fav_win = pd.read_csv(r'.\Fav_Win_Count_Clay.csv')
dog_win = pd.read_csv(r'.\Dog_Win_Count_Clay.csv')

# Keep only necessary columns to avoid duplicates or unnecessary data
fav_win_subset = fav_win[['Fav', 'Fav_Wins_Last_30_Days']].drop_duplicates(subset='Fav')
dog_win_subset = dog_win[['Dog', 'Dog_Wins_Last_30_Days']].drop_duplicates(subset='Dog')

# Merge based only on player names
date_filter = date_filter.merge(fav_win_subset, on='Fav', how='left')
date_filter = date_filter.merge(dog_win_subset, on='Dog', how='left')
today=date_filter.copy()
today=today[today['Fav_Odds']>1.7]
today=today[today['Dog_Wins_Last_30_Days']<today['Fav_Wins_Last_30_Days']]
mens_favs=today[today['Sex']!='Womens']
mens_favs[['Time','Dog','Dog_Odds','Fav','Fav_Odds','Dog_Wins_Last_30_Days','Fav_Wins_Last_30_Days']]

,Time,Dog,Dog_Odds,Fav,Fav_Odds,Dog_Wins_Last_30_Days,Fav_Wins_Last_30_Days
4,02:05,Facundo Diaz Acosta,1.96,Thiago Agustin Tirante,1.77,0.0,6.0
25,22:00,Lorenzo Carboni,1.97,Gonzalo Villanueva,1.75,0.0,3.0


In [281]:
# Current date and time
current_datetime = datetime.datetime.now()#+ datetime.timedelta(days=-1)  
# Convert to string
date_string = current_datetime.strftime("%Y-%m-%d")

mens['Date']=date_string
mens.to_sql('Preditions_Clay',con=devengine,if_exists='append',index=False)

womens['Date']=date_string
womens.to_sql('Preditions_Clay',con=devengine,if_exists='append',index=False)

1

In [284]:
predictions=pd.read_sql_query('Select Distinct * from Preditions_Clay where date > "2025-07-04"',con=devengine)
results=temp
stake=100

# Ensure date types
predictions['Date'] = pd.to_datetime(predictions['Date'])
results['Date'] = pd.to_datetime(results['Date'])

# Step 1: Initial merge on the exact date
merged_df = pd.merge(
    predictions, 
    results[['Date', 'Fav', 'Dog', 'Winner']],
    on=['Date', 'Fav', 'Dog'], 
    how='left'
)

# Step 2: Get unmatched rows only
unmatched = merged_df[merged_df['Winner'].isna()].copy()

if not unmatched.empty:
    # Try matching with results where match happened one day after the prediction date
    results_shifted = results.copy()
    results_shifted['Date'] = results_shifted['Date'] - pd.Timedelta(days=1)

    fallback = pd.merge(
        unmatched.drop(columns=['Winner']),  # remove old (NaN) winner
        results_shifted[['Date', 'Fav', 'Dog', 'Winner']],
        on=['Date', 'Fav', 'Dog'],
        how='left'
    )

    # Only update original merged_df where Winner was missing AND fallback found a match
    merged_df.loc[merged_df['Winner'].isna(), 'Winner'] = fallback['Winner'].values


# merged_df now has best effort match: same day, or next day fallback
merged_df_filtered=merged_df
#merged_df_filtered=merged_df[merged_df['Winner'].notna()].copy()
#merged_df_filtered=merged_df_filtered[merged_df_filtered['Sex']=='Womens']


merged_df_filtered['Profit'] = merged_df_filtered.apply(    lambda row: (row['Dog_Odds'] * stake)-stake if row['Winner'] == row['Dog'] else -stake, axis=1
)

# Calculate total profit
total_profit = merged_df_filtered['Profit'].sum()

# Print the total profit
print("Total Profit:", total_profit, len(merged_df_filtered),len(merged_df_filtered[merged_df_filtered['Dog']==merged_df_filtered['Winner']])/len(merged_df_filtered))
merged_df_filtered[['Time','Date','Dog_Odds','Winner','Dog','Fav','Profit','Dog_Wins_Last_30_Days','Fav_Wins_Last_30_Days']]

Total Profit: 69.0 14 0.5


,Time,Date,Dog_Odds,Winner,Dog,Fav,Profit,Dog_Wins_Last_30_Days,Fav_Wins_Last_30_Days
0,20:00,2025-07-06,1.93,Francesco Maestrelli,Francesco Maestrelli,Luka Pavlovic,93.0,1,0
1,22:00,2025-07-08,1.90,Francesco Passaro,Francesco Passaro,Luca Van Assche,90.0,5,0
2,20:00,2025-07-08,2.27,Dalma Galfi,Dalma Galfi,Kaja Juvan,127.0,1,0
3,20:00,2025-07-10,1.97,Titouan Droguet,Titouan Droguet,Hugo Dellien,97.0,3,0
4,21:30,2025-07-10,2.04,Juan Manuel Cerundolo,Raphael Collignon,Juan Manuel Cerundolo,-100.0,2,0
5,20:55,2025-07-10,1.97,Titouan Droguet,Titouan Droguet,Hugo Dellien,97.0,3,0
6,22:45,2025-07-10,2.10,Antonia Ruzic,Antonia Ruzic,Lucia Bronzetti,110.0,1,0
7,20:30,2025-07-11,2.13,Petra Marcinko,Lola Radivojevic,Petra Marcinko,-100.0,1,0
8,23:45,2025-07-11,2.08,Elsa Jacquemot,Simona Waltert,Elsa Jacquemot,-100.0,1,0
9,00:30,2025-07-12,2.16,Elsa Jacquemot,Simona Waltert,Elsa Jacquemot,-100.0,1,0


In [285]:
#df=pd.read_clipboard()
#df['Date'] = pd.to_datetime(df['Date'], errors='coerce')
#df['Date'] = df['Date'].dt.strftime('%Y-%m-%d')
#df.to_sql('Preditions_Clay',con=devengine,if_exists='replace',index=False)



#pd.read_sql_query('Select * from Preditions_Clay',con=devengine)#.to_clipboard(index=False)


df=pd.read_sql_query('Select * from TomorrowsMatches',con=devengine)#.to_clipboard(index=False)
df["Fav_Odds"] = df.apply(
    lambda row: min(row["Player_1_Odds"], row["Player_2_Odds"]), axis=1
)
df["Dog_Odds"] = df.apply(
    lambda row: max(row["Player_1_Odds"], row["Player_2_Odds"]), axis=1
)
# Create columns for Fav and Dog based on odds
df["Fav"] = df.apply(
    lambda row: row["Player_1"] if row["Player_1_Odds"] == row["Fav_Odds"] else row["Player_2"], axis=1
)
df["Dog"] = df.apply(
    lambda row: row["Player_2"] if row["Player_1_Odds"] == row["Fav_Odds"] else row["Player_1"], axis=1
)
date_filter = df[df['Surface']=='Clay']
fav_win = pd.read_csv(r'.\Fav_Win_Count_Clay.csv')
dog_win = pd.read_csv(r'.\Dog_Win_Count_Clay.csv')

# Keep only necessary columns to avoid duplicates or unnecessary data
fav_win_subset = fav_win[['Fav', 'Fav_Wins_Last_30_Days']].drop_duplicates(subset='Fav')
dog_win_subset = dog_win[['Dog', 'Dog_Wins_Last_30_Days']].drop_duplicates(subset='Dog')

# Merge based only on player names
date_filter = date_filter.merge(fav_win_subset, on='Fav', how='left')
date_filter = date_filter.merge(dog_win_subset, on='Dog', how='left')
date_filter[['Date','Time','Sex','Surface','Tournament','Fav','Dog','Fav_Odds','Dog_Odds','Fav_Wins_Last_30_Days','Dog_Wins_Last_30_Days']]

,Date,Time,Sex,Surface,Tournament,Fav,Dog,Fav_Odds,Dog_Odds,Fav_Wins_Last_30_Days,Dog_Wins_Last_30_Days
0,2025-07-12,01:00,Mens,Clay,Iasi challenger16,Stan Wawrinka,Radu Albot,1.29,3.06,2,4
1,2025-07-12,02:30,Mens,Clay,Iasi challenger16,Elmer Moller,Francesco Maestrelli,1.40,2.76,2,8
2,2025-07-12,01:00,Mens,Clay,Trieste challenger21,Facundo Diaz Acosta,Stefano Travaglia,1.33,2.81,3,8
3,2025-07-12,01:30,Mens,Clay,Trieste challenger21,Thiago Agustin Tirante,Santiago Rodriguez Taverna,1.28,3.50,6,1
4,2025-07-12,01:30,Womens,Clay,Contrexeville WTA13,Varvara Gracheva,Oksana Selekhmeteva,1.58,2.28,1,2


In [287]:
df=pd.read_sql_query('Select * from TodaysMatches',con=devengine)#.to_clipboard(index=False)
df["Fav_Odds"] = df.apply(
    lambda row: min(row["Player_1_Odds"], row["Player_2_Odds"]), axis=1
)
df["Dog_Odds"] = df.apply(
    lambda row: max(row["Player_1_Odds"], row["Player_2_Odds"]), axis=1
)
# Create columns for Fav and Dog based on odds
df["Fav"] = df.apply(
    lambda row: row["Player_1"] if row["Player_1_Odds"] == row["Fav_Odds"] else row["Player_2"], axis=1
)
df["Dog"] = df.apply(
    lambda row: row["Player_2"] if row["Player_1_Odds"] == row["Fav_Odds"] else row["Player_1"], axis=1
)
date_filter = df[df['Surface']=='Hard']
fav_win = pd.read_csv(r'.\Fav_Win_Count_Hard.csv')
dog_win = pd.read_csv(r'.\Dog_Win_Count_Hard.csv')

# Keep only necessary columns to avoid duplicates or unnecessary data
fav_win_subset = fav_win[['Fav', 'Fav_Wins_Last_30_Days']].drop_duplicates(subset='Fav')
dog_win_subset = dog_win[['Dog', 'Dog_Wins_Last_30_Days']].drop_duplicates(subset='Dog')

# Merge based only on player names
date_filter = date_filter.merge(fav_win_subset, on='Fav', how='left')
date_filter = date_filter.merge(dog_win_subset, on='Dog', how='left')
date_filter[['Date','Time','Sex','Surface','Tournament','Fav','Dog','Fav_Odds','Dog_Odds','Fav_Wins_Last_30_Days','Dog_Wins_Last_30_Days']]
df

KeyError: "['Fav_Wins_Last_30_Days'] not in index"